## Libraries and Exploration

In [1]:
import sys
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install numpy
#!{sys.executable} -m pip install nltk
#!{sys.executable} -m pip install matplotlib
#!{sys.executable} -m pip install sklearn
#!{sys.executable} -m pip install WordCloud
#!{sys.executable} -m pip install selenium
#!{sys.executable} -m pip install gensim
#!{sys.executable} -m pip install stanfordnlp
#!{sys.executable} -m pip install polyglot
#!{sys.executable} -m pip install pyLDAvis
#!{sys.executable} -m pip install TextBlob

In [2]:
import numpy as np
import pandas as pd
from IPython.display import display
from tqdm import tqdm
from collections import Counter
import ast

import arabic_reshaper
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sb

from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob
import scipy.stats as stats

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
output_notebook()

%matplotlib inline

Loading BokehJS ...

In [3]:
datafile = '..\\woman-opinion.csv'
raw_data = pd.read_csv(datafile, parse_dates=[0], infer_datetime_format=True)

reindexed_data = raw_data['concordance']
reindexed_data.index = raw_data['date']

raw_data.head()

,date,concordance
0,1996-01-01,كتابته عن المرأة واللغة نموذجا لتوضيح كتاب الم...
1,1999-01-01,بالاضافة الى ذلك تحتاج المرأة وخاصة الشابة وال...
2,1999-01-01,الجماع لشكوى المرأة وايضا لشكوى الرجل
3,1999-01-01,بين المرأة المسلمة والمرأة في أوروبا
4,1999-01-01,الأجنبيات ومحاولة استمالة المرأة لفعل


from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stopwords_list = stopwords.words('function-ar')
print(stopwords_list)

def get_stopwords():
    nltk_stopwords = set(stopwords.words('function-ar'))
    sklearn_stopwords = stop_words.ARABIC_STOP_WORDS

    all_stopwords = set()
    all_stopwords |= spacy_stopwords
    all_stopwords |= nltk_stopwords
    all_stopwords |= sklearn_stopwords

    return all_stopwords

In [4]:
# Define helper functions
def get_top_n_words(n_top_words, count_vectorizer, text_data):
    '''
    returns a tuple of the top n words in a sample and their 
    accompanying counts, given a CountVectorizer object and text sample
    '''
    vectorized_concordances = count_vectorizer.fit_transform(text_data.values)
    vectorized_total = np.sum(vectorized_concordances, axis=0)
    word_indices = np.flip(np.argsort(vectorized_total)[0,:], 1)
    word_values = np.flip(np.sort(vectorized_total)[0,:],1)
    
    word_vectors = np.zeros((n_top_words, vectorized_concordances.shape[1]))
    for i in range(n_top_words):
        word_vectors[i,word_indices[0,i]] = 1

    words = [word[0].encode('utf-8').decode('utf-8') for 
             word in count_vectorizer.inverse_transform(word_vectors)]

    return (words, word_values[0,:n_top_words].tolist()[0])

In [5]:
tagged_concordances = [TextBlob(reindexed_data[i]).pos_tags for i in range(reindexed_data.shape[0])]

In [6]:
tagged_concordances_df = pd.DataFrame({'tags':tagged_concordances})

word_counts = [] 
pos_counts = {}

for concordance in tagged_concordances_df[u'tags']:
    word_counts.append(len(concordance))
    for tag in concordance:
        if tag[1] in pos_counts:
            pos_counts[tag[1]] += 1
        else:
            pos_counts[tag[1]] = 1
            
print('Total number of words: ', np.sum(word_counts))
print('Mean number of words per concordance: ', np.mean(word_counts))

Total number of words:  439333
Mean number of words per concordance:  24.56707487558016


In [9]:
# NLTK Stop words

mylist= ['إذ','إذا','إذما','إذن','أف','أقل','أكثر','ألا','إلا','التي','الذي','الذين','اللاتي','اللائي','اللتان','اللتيا','اللتين','اللذان','اللذين','اللواتي','إلى','إليك','إليكم','إليكما','إليكن','أم','أما','أما','إما','أن','إن','إنا','أنا','أنت','أنتم','أنتما','أنتن','إنما','إنه','أنى','أنى','آه','آها','أو','أولاء','أولئك','أوه','آي','أي','أيها','إي','أين','أين','أينما','إيه','بخ','بس','بعد','بعض','بك','بكم','بكم','بكما','بكن','بل','بلى','بما','بماذا','بمن','بنا','به','بها','بهم','بهما','بهن','بي','بين','بيد','تلك','تلكم','تلكما','ته','تي','تين','تينك','ثم','ثمة','حاشا','حبذا','حتى','حيث','حيثما','حين','خلا','دون','ذا','ذات','ذاك','ذان','ذانك','ذلك','ذلكم','ذلكما','ذلكن','ذه','ذو','ذوا','ذواتا','ذواتي','ذي','ذين','ذينك','ريث','سوف','سوى','شتان','عدا','عسى','عل','على','عليك','عليه','عما','عن','عند','غير','فإذا','فإن','فلا','فمن','في','فيم','فيما','فيه','فيها','قد','كأن','كأنما','كأي','كأين','كذا','كذلك','كل','كلا','كلاهما','كلتا','كلما','كليكما','كليهما','كم','كم','كما','كي','كيت','كيف','كيفما','لا','لاسيما','لدى','لست','لستم','لستما','لستن','لسن','لسنا','لعل','لك','لكم','لكما','لكن','لكنما','لكي','لكيلا','لم','لما','لن','وبما','أنهم','إنهم','فبما','لنا','له','لها','لهم','لهما','لهن','لو','لولا','لوما','لي','لئن','ليت','ليس','ليسا','ليست','ليستا','ليسوا','ما','ماذا','متى','مذ','مع','مما','ممن','من','منه','منها','منذ','مه','مهما','نحن','نحو','نعم','ها','هاتان','هاته','هاتي','هاتين','هاك','هاهنا','هذا','هذان','هذه','هذي','هذين','هكذا','هل','هلا','هم','هما','هن','هنا','هناك','هنالك','هو','هؤلاء','هي','هيا','هيت','هيهات','والذي','والذين','وإذ','وإذا','وإن','ولا','ولكن','ولو','وما','ومن','وهو','يا','هي','أفهي','أوهي','فهي','وهي','لهي','هو','أهو','أوهو','أفهو','فهو','وهو','لهو','ولهو','فلهو','هم','أهم','أفهم','أوهم','بهم','فهم','وهم','لهم','ولهم','فلهم','هن','أهن','أوهن','أفهن','فهن','بهن','وهن','لهن','فلهن','ولهن','هما','فهما','لهما','وهما','بهما','ولبهما','فلبهما','أنت','وأنت','لأنت','فأنت','ولأنت','فلأنت','أنتم','وأنتم','لأنتم','فأنتم','فلأنتم','ولأنتم','أنت','وأنتن','لأنتن','فأنتن','فلأنتن','ولأنتن','أنتما','وأنتما','لأنتما','فأنتما','ولأنتما','فلأنتما','نحن','أفنحن','أونحن','أولنحن','ونحن','فنحن','لنحن','ولنحن','فلنحن','أنا','أأنا','أفأنا','أوأنا','وأنا','لأنا','فأنا','ولأنا','فلأنا','هذا','أهذا','أفهذا','أوهذا','بهذا','وهذا','كهذا','لهذا','فهذا','ولهذا','فلهذا','هذه','أهذه','أفهذه','أوهذه','بهذه','وهذه','كهذه','لهذه','فهذه','ولهذه','فلهذه','هذان','أهذان','أفهذان','أوهذان','بهذين','وهذان','كهذين','لهذين','فهذان','ولهذين','فلهذان','هذين','أهذين','أفهذين','أوهذين','بهذين','وهذين','كهذين','لهذان','فهذين','ولهذين','فلهذين','هؤلاء','أهؤلاء','فهؤلاء','وهؤلاء','ولهؤلاء','وبهؤلاء','أفهؤلاء','أوهؤلاء','لهؤلاء','كهؤلاء','بهؤلاء','أكهؤلاء','هاتان','أهاتان','أفهاتان','أوهاتان','وهاتان','كهاتين','لهاتين','فهاتان','ولهاتين','فلهاتان','هاتين','أهاتين','أفهاتين','أوهاتين','بهاتين','وهاتين','كهاتين','لهاتان','فهاتين','ولهاتين','فلهاتين','الذين','اللذان','الذي','التي','اللذين','اللاتي','اللائي','والذين','واللذان','والذي','والتي','واللذين','واللاتي','واللائي','فالذين','فاللذان','فالذي','فالتي','فاللذين','فاللاتي','فاللائي','كالذين','كاللذان','كالذي','كالتي','كاللذين','كاللاتي','كاللائي','للذين','لللذان','للذي','للتي','للذين','للاتي','للائي','بالذين','باللذان','بالذي','بالتي','باللذين','باللاتي','باللائي','أوالذين','أواللذان','أوالذي','أوالتي','أواللذين','أواللاتي','أواللائي','أفالذين','أفاللذان','أفالذي','أفالتي','أفاللذين','أفاللاتي','أفاللائي','وللذين','ولللذان','وللذي','وللتي','وللذين','وللاتي','وللائي','وبالذين','وباللذان','وبالذي','وبالتي','وباللذين','وباللاتي','وباللائي','ثم','فثمة','وثمة','لم','ولم','فلم','سوف','وسوف','فسوف','أفسوف','أوسوف','ولسوف','أن','إن','إنا','أنا','وأن','وإن','وإنا','وأنا','فأن','فإن','فإنا','فأنا','بأن','بإن','بإنا','بأنا','أوأن','أوإن','أوإنا','أوأنا','ولأن','ولإن','ولإنا','ولأنا','لئن','فلئن','ولئن','أولئن','أفلئن','ما','وما','أفما','أوما','ولما','لما','كلما','فلما','عم','عمن','بما','فما','كما','وكما','فكما','قد','وقد','لقد','فقد','لكن','ولكن','فلكن','لكنهم','ولكنهم','فلكنهم','لكنه','ولكنه','فلكنه','لكنها','ولكنها','فلكنها','لكنهما','ولكنهما','فلكنهما','لكنهن','ولكنهن','فلكنهن','إذا','إذ','فإذا','وإذا','فإذ','وإذ','أو','في','وفي','ففي','لفي','وفي','من','ومن','فمن','لمن','أومن','أفمن','كمن','إلى','وإلى','فإلى','أإلى','إلام','علام','حتام','عن','وعن','فعن','أوعن','طالما','لطالما','فطالما','حاشا','وحاشا','وعدا','إلا','فحاشا','فعدا','وإلا','ألا','كلا','نعم','اللهم','كل','فكل','ككل','ولكل','وككل','وكل','لكل','ولكل','فلكل','بكل','وبكل','فبكل','كلهم','فكلهم','ككلهم','ولكلهم','وككل','وكلهم','لكلهم','ولكلهم','فلكلهم','بكلهما','وبكلهما','فبكلهما','كلهن','فكلهن','ككل','ولكل','وككل','وكلكم','لكلكم','ولكلكم','فلكلكم','بكل','وبكل','فبكل','كلهما','وكلهما','فكلهما','فكلهن','كله','وكله','فكله','بكله','لكله','ككله','كلها','وكلها','بكلها','فكلها','لكلها','ككلها','ولكلها','بعض','ببعض','وببعض','فببعض','وبعض','ولبعض','فلبعض','فبعض','كبعض','وكبعض','بعضها','ببعضها','وببعضها','فببعضها','وبعضها','ولبعضها','فلبعضها','فبعضها','كبعضها','وكبعضها','بعضه','ببعضه','وببعضه','فببعضه','وبعضه','ولبعضه','فلبعضه','فبعضه','كبعضه','وكبعضه','بعضهن','ببعضهن','وببعضهن','فببعضهن','وبعضهن','ولبعضهن','فلبعضهن','فبعضهن','كبعضهن','وكبعضهن','بعضهم','ببعضهم','وببعضهم','فببعضهم','وبعضهم','ولبعضهم','فلبعضهم','فبعضهم','كبعضهم','وكبعضهم','بعضهما','ببعضهما','وببعضهما','فببعضهما','وبعضهما','ولبعضهما','فلبعضهما','فبعضهما','كبعضهما','وكبعضهما','جميع','وجميع','فجميع','لجميع','كجميع','جميعهم','وجميعهم','فجميعهم','بجميع','بجميعهم','جميعهن','وجميعهن','ولجميعهم','بجميعهن','كجميعهن','جميعها','وجميعها','فجميعها','لجميعها','كجميعها','جميعه','وجميعه','فجميعه','لجميعه','كجميعه','إلخ','حيث','وحيث','بحيث','ثم','فثم','فثمة','ليس','وليس','لا','ولا','ألا','إلا','كلا','نعم','بلى','أفلا','أولا','قبل','قبله','قبلها','قبلهن','قبلهم','قبلي','قبلك','قبلكم','قبلهما','قبلكما','وقبل','وقبله','وقبلها','وقبلهن','وقبلهم','وقبلي','وقبلك','وقبلكم','وقبلهما','وقبلكما','فقبل','فقبله','فقبلها','فقبلهن','فقبلهم','فقبلي','فقبلك','فقبلكم','فقبلهما','فقبلكما','ولقبلكما','بعد','بعده','بعدها','بعدهن','بعدهم','بعدي','بعدك','بعدكم','بعدهما','بعدكما','وبعد','وبعده','وبعدها','وبعدهن','وبعدهم','وبعدي','وبعدك','وبعدكم','وبعدهما','وبعدكما','فبعد','فبعده','فبعدها','فبعدهن','فبعدهم','ولبعدهم','فبعدي','فبعدك','فبعدكم','فبعدهما','فبعدكما','ولبعدكما','من','منه','منها','منهما','منكما','منهم','منهن','منك','مني','منكن','ومن','ومنه','ومنها','ومنهما','ومنكما','ومنهم','ومنهن','ومنك','ومني','ومنكن','فمن','فمنه','فمنها','فمنهما','فمنكما','فمنهم','فمنهن','فمنك','فمني','فمنكن','أمن','أمنه','أمنها','أمنهما','أمنكما','أمنهم','أمنهن','أمنك','أمني','أمنكن','عن','عنه','عنها','عنهما','عنكما','عنهم','عنهن','عنك','عني','عنكن','وعن','وعنه','وعنها','وعنهما','وعنكما','وعنهم','وعنهن','وعنك','وعني','وعنكن','فعن','فعنه','فعنها','فعنهما','فعنكما','فعنهم','فعنهن','فعنك','فعني','فعنكن','أعن','أعنه','أعنها','أعنهما','أعنكما','أعنهم','أعنهن','أعنك','أعني','أعنكن','قد','وقد','فلقد','ولقد','لم','ولم','أولم','فلم','ل','له','لها','لهما','لكما','لهم','لهن','لك','لي','لكن','ول','وله','ولها','ولهما','ولكما','ولهم','ولهن','ولك','ولي','ولكن','فل','فله','فلها','فلهما','فلكما','فلهم','فلهن','فلك','فلي','فلكن','أل','أله','ألها','ألهما','ألكما','ألهم','ألهن','ألك','ألي','ألكن','في','فيه','فيها','فيهما','فيكما','فيهم','فيهن','فيك','فيي','فيكن','وفي','وفيه','وفيها','وفيهما','وفيكما','وفيهم','وفيهن','وفيك','وفيي','وفيكن','ففي','ففيه','ففيها','ففيهما','ففيكما','ففيهم','ففيهن','ففيك','ففيي','ففيكن','أفي','أفيه','أفيها','أفيهما','أفيكما','أفيهم','أفيهن','أفيك','أفيي','أفيكن','عند','عنده','عندها','عندهما','عندكما','عندهم','عندهن','عندك','عندي','عندكن','وعند','وعنده','وعندها','وعندهما','وعندكما','وعندهم','وعندهن','وعندك','وعندي','وعندكن','فعند','فعنده','فعندها','فعندهما','فعندكما','فعندهم','فعندهن','فعندك','فعندي','فعندكن','أعند','أعنده','أعندها','أعندهما','أعندكما','أعندهم','أعندهن','أعندك','أعندي','أعندكن','مع','معه','معها','معهما'
         ,'معكما','معهم','معهن','معك','معي','معكن','ومع','ومعه','ومعها','ومعهما','ومعكما','ومعهم','ومعهن','ومعك','ومعي','ومعكن','فمع','فمعه','فمعها','فمعهما','فمعكما','فمعهم','فمعهن','فمعك','فمعي','فمعكن','أمع','أمعه','أمعها','أمعهما','أمعكما','أمعهم','أمعهن','أمعك','أمعي','أمعكن','إلى','إليه','إليها','إليهما','إليكما','إليهم','إليهن','إليك','إليي','إليكن','وإلي','وإليه','وإليها','وإليهما','وإليكما','وإليهم','وإليهن','وإليك','وإليكن','فإلي','فإليه','فإليها','فإليهما','فإليكما','فإليهم','فإليهن','فإليك','فإليي','فإليكن','أإلي','أإليه','أإليها','أإليهما','أإليكما','أإليهم','أإليهن','أإليك','أإليي','أإليكن','على','عليه','عليها','عليهما','عليكما','عليهم','عليهن','عليك','عليكن','وعلى','وعليه','وعليها','وعليهما','وعليكما','وعليهم','وعليهن','وعليك','وعليي','وعليكن','فعلى','فعليه','فعليها','فعليهما','فعليكما','فعليهم','فعليهن','فعليك','فعليي','فعليكن','أعليه','أعليها','أعليهما','أعليكما','أعليهم','أعليهن','أعليك','أعليكن','إياك','فإياك','وإياك','إياكم','فإياكم','وإياكم','إياكن','فإياكن','وإياكن','إياكما','فإياكما','وإياكما','إياكم','فإياكم','وإياكم','إياهم','فإياهم','وإياهم','إياهن','فإياهن','وإياهن','إياي','فإياي','وإياي','إيانا','فإيانا','وإيانا','كذلك','وكذلك','ولذلك','تلكم','وتلكم','وتلكما','فتلكما','ربما','ولربما','فلربما','وربما','لعل','ولعل','فلعل','ليت','وليت','فليت','يا','أو','لو','لولا','ولو','فلو','فلولا','ولولا','ب','به','بها','بهما','بكما','بهم','بهن','بك','بي','بكن','وب','وبه','وبها','وبهما','وبكما','وبهم','وبهن','وبك','وبي','وبكن','فب','فبه','فبها','فبهما','فبكما','فبهم','فبهن','فبك','فبي','فبكن','أبه','أبها','أبهما','أبكما','أبهم','أبهن','أبك','أبي','أبكن','بين','وبين','فبين','لبين','بينه','وبينه','فبينه','لبينه','بينها','بينهما','وبينهما','فبينهما','ولبينهما','لبينهما','وبينهم','فبينهم','ولبينهم','لبينهم','وبينها','فبينها','ولبينها','لبينها','بينهم','بينهن','لبينهن','وبينهن','فبينهن','بيني','وبيني','فبيني','لبيني','بيننا','فبيننا','وبيننا','لبيننا','خلال','خلالهما','خلالهن','خلاله','خلالها','خلالي','خلالك','خلالنا','وخلال','وخلالهما','وخلالهن','وخلاله','وخلالها','وخلالي','وخلالك','وخلالنا','فخلال','فخلالهما','فخلالهن','فخلاله','فخلالها','فخلالي','فخلالك','فخلالنا','تحت','وتحت','فتحت','ولتحت','تحته','وتحته','فتحته','ولتحته','تحتها','وتحتها','فتحتها','ولتحتها','تحتهم','وتحتهم','فتحتهم','ولتحتهم','تحتهن','وتحتهن','فتحتهن','ولتحتهن','تحتي','وتحتي','فتحتي','ولتحتي','تحتهما','وتحتهما','فتحتهما','ولتحتهما','تحتكم','وتحتكم','فتحتكم','ولتحتكم','تحتكن','وتحتكن','فتحتكن','ولتحتكن','تحتكما','وتحتكما','فتحتكما','ولتحتكما','فوق','وفوق','ففوق','ولفوق','فوقه','وفوقه','ففوقه','ولفوقه','فوقها','وفوقها','ففوقها','ولفوقها','فوقهم','وفوقهم','ففوقهم','ولفوقهم','فوقهن','وفوقهن','ففوقهن','ولفوقهن','فوقي','وفوقي','ففوقي','ولفوقي','فوقهما','وفوقهما','ففوقهما','ولفوقهما','فوقكم','وفوقكم','ففوقكم','ولفوقكم','فوقكن','وفوقكن','ففوقكن','ولفوقكن','فوقكما','وفوقكما','ففوقكما','ولفوقكما','أ','ب','ت','ث','ج','ح','خ','د','ذ','ر','ز','س','ش','ص','ض','ط','ظ','ع','غ','ف','ق','ك','ل','م','ن','ه','و','ي','أي','أيهم','أيها','أيهن','أينا','أيكم','أيهم','أيكن','أيهن','وأي','وأيهم','وأيها','وأيهن','وأينا','وأيكم','وأيهم','وأيكن','وأيهن','فأي','فأيهم','فأيها','فأيهن','فأينا','فأيكم','فأيهم','فأيكن','فأيهن','لأي','لأيهم','لأيها','لأيهن','لأينا','لأيكم','لأيهم','لأيكن','لأيهن','كأي','كأيهم','كأيها','كأيهن','كأينا','كأيكم','كأيهم','كأيكن','كأيهن','ذلك','وذلك','فذلك','أذلك','كذلك','بذلك','لذلك','ذلكم','وذلكم','فذلكم','أذلكم','كمذلكم','بذلكم','لذلكم','ذلكن','وذلكن','فذلكن','أذلكن','كنذلكن','بذلكن','لذلكن','ذلكما','وذلكما','فذلكما','أذلكما','كماذلكما','بذلكما','لذلكما','أن','لأن','لأنهم','لأنهما','لأنكم','لأني','لأنكما','لأنكن','لأني','وأن','ولأن','ولأنهم','ولأنهما','ولأنكم','ولأني','ولأنكما','ولأنكن','ولأني','فأن','فلأن','فلأنهم','فلأنهما','فلأنكم','فلأني','فلأنكما','فلأنكن','فلأني','فلئن','ولئن','فلا','لأنه','بل','مما','ذا','كذا','هل','وهل','فهل','أنه','أنها','أنهم','أنهن','أنهما','أنني','أنكم','أننا','وأنه','وأنها','وأنهم','وأنهن','وأنهما','وأنني','وأنكم','وأننا','فأنه','فأنها','فأنهم','فأنهن','فأنهما','فأنني','فأنكم','فأننا','كأنه','كأنها','كأنهم','كأنهن','كأنهما','كأنني','كأنكم','كأننا','لأنه','لأنها','لأنهم','لأنهن','لأنهما','لأنني','لأنكم','لأننا','أني','وأني','فأني','كأني','لنا','ولنا','فلنا','حتى','وحتى','فحتى','فيما','أما','وأما','إنما','أم','وأما','إما','ثنا','حين','وحين','فحين','يومئذ','ويومئذ','فيومئذ','عندئذ','وعندئذ','فعندئذ','إني','هنا','هناك','وهناك','فهناك','لهناك','عما','وعما','وعم','عم','لما','أنى','أيان' ,'ماذا','لماذا','ولماذا','فلماذا','وعلام','وحتام','طالما','قلما','وطالما','فطالما','وقلما','ذاك','ذينك','تينك','ذينك','ليست','وليست','فليست','أليست','أوليست','أوليس','فليس','وليس','إنها','بينه','كيف','وكيف','فكيف','لكيف','لن','فلن','ولن','ألن','أنهما','إذن','أن','أوربما','أربما','منذ','كأن' ,'فإنك','هكذا','وهكذا','فإنه','فإنها','فإنهم','فإنهن','فإنهما','فإنك','فإنهما','فإنكما','فإنكم','فإني','فإنني','الا','اذا','اذ','الى','فى','آل','الدكتور' ,'السعودية','العربية','عبدالله','سعود','خلال','متي','ومتى','متى','فمتى','فأيان','كمن','فيمن','فأما','نا','ثنا','ومما' ,'فإننا','فإنا','كان','فكان','وكان','يكون','كانوا','كنا','كنتم','وكنتم','فكنا','وكنا','لكنا','ولكنا','كنت','وكنت','وعندنا','عندنا','اه','أنى','عا' ,'انه','انها','كذا','وكذا','فكذا','ال','وهكذا','ؤ','ء','لآ','آ','ا','ئ','ة','لإ','ى','هكذا','لهذا','فلهذا','ولهذا','كانت','إذا','إذما','إذن' ,'أف','أقل','أكثر','ألا','إلا','التي','الذي','الذين','اللاتي','اللائي','اللتان','اللتيا','اللتين','اللذان','اللذين','اللواتي','إلى','إليك','إليكم','إليكما' ,'إليكن','أم','أما','أما','إما','أن','إن','إنا','أنا','أنت','أنتم','أنتما','أنتن','أنما','إنه','أنى','أنى','آه','آها','أو','أولاء','أولئك','أوه' ,'آي','أي','أيها','إي','أين','أين','أينما','إيه','بخ','بس','بعد','بعض','بك','بكم','بكم','بكما','بكن','بل','بلى','بما','بماذا','بمن','بنا','عدد','تم','كورونا','العام' ,'به','بها','بهم','بهما','بهن','بي','بين','بيد','تلك','تلكم','تلكما','ته','تي','تين','تينك','ثم','ثمة','حاشا','حبذا','حتى','حيث','حيثما' ,'حين','خلا','دون','ذا','ذات','ذاك','ذان','ذانك','ذلك','ذلكم','ذلكما','ذلكن','ذه','ذو','ذوا','ذواتا','ذواتي','ذي','ذين','ذينك','ريث' ,'سوف','سوى','شتان','عدا','عسى','عل','على','عليك','عليه','عما','عن','عند','غير','فإذا','فإن','فلا','فمن','في','فيم','فيما','فيه' ,'فيها','قد','كأن','كأنما','كأي','كأين','كذا','كذلك','كل','كلا','كلاهما','كلتا','كلما','كليكما','كليهما','كم','كم','كما','كي','كيت','كيف' ,'كيفما','لا','لاسيما','لدى','لست','لستم','لستما','لستن','لسن','لسنا','لعل','لك','لكم','لكما','لكن','لكنما','لكي','لكيلا','لم','لما','لن','لنا','له' ,'لها','لهم','لهما','لهن','لو','لولا','لوما','لي','لئن','ليت','ليس','ليسا','ليست','ليستا','ليسوا','ما','ماذا','متى','مذ','مع','مما','ممن','من' ,'منه','منها','منذ','مه','مهما','نحن','نحو','نعم','ها','هاتان','هاته','هاتي','هاتين','هاك','هاهنا','هذا','هذان','هذه','هذي','هذين','هكذا','هل','هلا','هم','هما','هن','هنا','هناك','هنالك','هو','هؤلاء','هي','هيا','هيت','هيهات','والذي','فكانت','وإذ','وإذا','وإن','ولا','ولكن','يمكن','يتم','يجب','حول' ,'ولو','وما','ومن','وهو','يا','إذما','إذما','إذما','إذما','والذين','تكون','ان','ولكنتما','وكانتا','فكانتا','كانتا','فكنتم','فكنا','فكن','وكن','كن','وزير','الدكتور','خالد','فيصل','وقال','قال','عبر','فقط' ,'يكن','لتكن','ولتكن','فلتكن','ولكانت','لكانت','العام','كونوا','كنتما','عندما','الله','بن','سلمان','المملكة','عبدالعزيز','محمد','عبد','اليوم','يوم','أمس','أحد','أيضا','يوما','عام','عاما','مثل','البحر','الأحمر','المدينة','المنورة','مكة','المكرمة','السمو','الملكي','الأمير','الملك' ,'المرأة', 'الرجل', 'النساء', 'الرجال', 'والرجال', 'والنساء', 'والرجل', 'والمرأة', 'للرجال', 'للنساء', 'تجد', 'تجد', 'عامة', 'ايضا', 'صلى', 'الله', 'وسلم', 'أكثر' ,'تعرف', 'يقول', 'اي', 'تعالى', 'عليكم', 'ل', 'ا', 'وان', 'ـة', 'س', 'رويترز', 'كثيرا', 'الـ', 'احمد', 'القاهرة', 'أبو', 'باب', 'السلام', 'ورحمة', 'وبركاته', 'ال' ,'الـ', 'الآية', 'او', 'نفسها', 'شيء', 'ير', 'جاء', 'جانب', 'وكانت', 'ر', 'أكد', 'رغم', 'سورة', 'أقام', 'بسبب', 'قسم', 'فهد', 'بينما', 'أول', 'ألف', 'ثلاثة' ,'مرة', 'بـ', 'عددا', 'مدى', 'بشكل', 'وخاصة', 'رجل', 'امرأة', 'وتم', 'الكثير', 'عدم','قال','قالت','وخصوصا','أكثر' ,'جريدة', 'الكثير', 'فلسطين', 'بريدة', 'الرياض', 'بالرجل', 'برجل', 'بالمرأة', 'بالنساء', 'بمرأة', 'بنساء', 'حولها', 'كثير', 'مرات', 'سوباري' ,'الأول', 'الثاني', 'الثالث', 'الرابع', 'الخامس', 'السادس', 'السابع', 'الثامن', 'التاسع', 'العاشر', 'صحيفة', 'الجزيرة', 'ومها', 'العلي', 'وانه' ,'كبيرا', 'جديدا', 'الميمان', 'يعيني', 'هونج', 'كونج', 'عبدالكريم', 'الشمري', 'بأنه', 'حديث', 'الحديث', 'ساوث', 'الوقت', 'وقت', 'رمضان', 'الدكتورة', 'الدكتور', 'بالرياض' ,'السعودي', 'وقالت', 'إحدى', 'بعنوان', 'داخل', 'انما', 'ايضا', 'اولئك', 'الأمر']

from nltk.corpus import stopwords
stop_words = mylist
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [10]:
small_count_vectorizer = CountVectorizer(stop_words= mylist, max_features=40000)
small_text_sample = reindexed_data.sample(n=10000, random_state=0).values

print('Text before vectorization: {}'.format(small_text_sample[123]))

small_document_term_matrix = small_count_vectorizer.fit_transform(small_text_sample)

print('Text after vectorization: \n{}'.format(small_document_term_matrix[123]))

Tweet before vectorization: النساء والملز والحائر ولمدة ساعة كان البعض 
Tweet after vectorization: 
  (0, 4412)	1
  (0, 32861)	1
  (0, 31744)	1
  (0, 36380)	1
  (0, 19456)	1


In [11]:
n_topics = 10

### LSA

In [12]:
lsa_model = TruncatedSVD(n_components=n_topics)
lsa_topic_matrix = lsa_model.fit_transform(small_document_term_matrix)

In [13]:
# Define helper functions
def get_keys(topic_matrix):
    '''
    returns an integer list of predicted topic 
    categories for a given topic matrix
    '''
    keys = topic_matrix.argmax(axis=1).tolist()
    return keys

def keys_to_counts(keys):
    '''
    returns a tuple of topic categories and their 
    accompanying magnitudes for a given list of keys
    '''
    count_pairs = Counter(keys).items()
    categories = [pair[0] for pair in count_pairs]
    counts = [pair[1] for pair in count_pairs]
    return (categories, counts)

In [14]:
lsa_keys = get_keys(lsa_topic_matrix)
lsa_categories, lsa_counts = keys_to_counts(lsa_keys)

In [15]:
# Define helper functions
def get_top_n_words(n, keys, document_term_matrix, count_vectorizer):
    '''
    returns a list of n_topic strings, where each string contains the n most common 
    words in a predicted category, in order
    '''
    top_word_indices = []
    for topic in range(n_topics):
        temp_vector_sum = 0
        for i in range(len(keys)):
            if keys[i] == topic:
                temp_vector_sum += document_term_matrix[i]
        temp_vector_sum = temp_vector_sum.toarray()
        top_n_word_indices = np.flip(np.argsort(temp_vector_sum)[0][-n:],0)
        top_word_indices.append(top_n_word_indices)   
    top_words = []
    for topic in top_word_indices:
        topic_words = []
        for index in topic:
            temp_word_vector = np.zeros((1,document_term_matrix.shape[1]))
            temp_word_vector[:,index] = 1
            the_word = count_vectorizer.inverse_transform(temp_word_vector)[0][0]
            topic_words.append(the_word.encode('utf-8').decode('utf-8'))
        top_words.append(" ".join(topic_words))        
    return top_words

In [16]:
top_n_words_lsa = get_top_n_words(10, lsa_keys, small_document_term_matrix, small_count_vectorizer)

for i in range(len(top_n_words_lsa)):
    print("Topic {}: ".format(i+1), top_n_words_lsa[i])

Topic 1:  والولادة أمراض والأطفال خاصة الحمل والتوليد استشاري الولادة حياة استشارية
Topic 2:  المجتمع دور المسلمة قيادة الحياة ضد للسيارة تعليم قضايا سواء
Topic 3:  الشعر شعر تساقط زيادة الإنسان الشاعر جزء خاصة الشاعرة الشعراء
Topic 4:  للسيارة قيادة بقيادة السيارة قادت السائق فاكهة تقطف السماح المتحفظين
Topic 5:  الحمل الحامل أثناء الحوامل فترة حدوث صحة الدم الجنين منع
Topic 6:  عمل العمل النسائية لعمل مجال مجالات تعمل الشباب بيع المجتمع
Topic 7:  سن الثدي نسبة الدورة سنة سرطان يصيب العمر اليأس الناس
Topic 8:  المسلمة عمل العمل الدعوة قرار حجاب الداعية الحجاب بحرية رأي
Topic 9:  حقوق الإسلام حق زوجها والطفل الإنسان وحقوق بحقوق الطفل خلقت
Topic 10:  الحامل التدخين البعض الأمراض تناول ظهور رحيما الخطأ تسبب السيئة


In [17]:
# Define helper functions
def get_mean_topic_vectors(keys, two_dim_vectors):
    '''
    returns a list of centroid vectors from each predicted topic category
    '''
    mean_topic_vectors = []
    for t in range(n_topics):
        articles_in_that_topic = []
        for i in range(len(keys)):
            if keys[i] == t:
                articles_in_that_topic.append(two_dim_vectors[i])    
        
        articles_in_that_topic = np.vstack(articles_in_that_topic)
        mean_article_in_that_topic = np.mean(articles_in_that_topic, axis=0)
        mean_topic_vectors.append(mean_article_in_that_topic)
    return mean_topic_vectors

### LDA

In [18]:
lda_model = LatentDirichletAllocation(n_components=n_topics, learning_method='online', 
                                          random_state=0, verbose=0)
lda_topic_matrix = lda_model.fit_transform(small_document_term_matrix)

In [19]:
lda_keys = get_keys(lda_topic_matrix)
lda_categories, lda_counts = keys_to_counts(lda_keys)

In [20]:
top_n_words_lda = get_top_n_words(10, lda_keys, small_document_term_matrix, small_count_vectorizer)

for i in range(len(top_n_words_lda)):
    print("Topic {}: ".format(i+1), top_n_words_lda[i])

Topic 1:  الشعر المسلمة الإسلام المجتمع خاصة حقوق الإنسان الزوج قضايا كتاب
Topic 2:  والولادة أمراض سن الحمل العمل عمل والتوليد استشاري استشارية العقم
Topic 3:  المجتمع تعليم عمل قضية حق للمرأة المسلمة مجتمعنا العمل التعليم
Topic 4:  الحب الحياة دموع المسلمة مكانة دائما الكريم علاقة المجتمع الإسلام
Topic 5:  والأطفال عمل النسائية الحياة العمل زوجها دور وجود المجتمع تريد
Topic 6:  قيادة للسيارة حقوق المجتمع السيارة عنوان العدد الأطفال قرأت رقم
Topic 7:  الآخر رئيس تحرير بنت المجتمع الشباب العمل تقول الوطن الأرض
Topic 8:  الثدي نسبة بسرطان سرطان خاصة حالة الإصابة العالم إصابة الدراسة
Topic 9:  الصالحة فتنة تقوم نجد شعر البيت كتابة أمام خاصة أخرى
Topic 10:  الحمل الحامل سن أثناء دور الحوامل المسلمة ضد فترة العنف
